# Import Packages

In [1]:
# -*- coding: utf-8 -*-
"""
Created on April 4, 2024

First Version Author: George Zhang


"""
User_Ackowledgement = input("If you are not the first author, please ADD comments with your name and date for ALL your edits. Ackowledge by answering Y", )

#if User_Ackowledgement is not Y, raise error

if User_Ackowledgement != 'Y':

    raise ValueError("You must acknowledge your edits by answering Y. Please add your name and date in the comments for your edits.")


#%%
from IPython import get_ipython
get_ipython().magic('reset -sf')

import sys
 
# appending a path
sys.path.append('')
 
import os
import pandas as pd
import numpy as np
import pickle
# import SFR_PD_Recalibration_2023_Lib as tool
# import Concentra_SFR_Fit_for_Use_2024_Lib as lib
from matplotlib import pyplot as plt
import math
import pyodbc

C:\Users\georgez\AppData\Local\Temp\ipykernel_20972\3988292044.py:20: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -sf')


# Setting Directories

In [2]:
current_dir = os.getcwd()  # 20250403 George added

# code_dir= "C:\\Users" +'\\' + username + '\\' + "Equitable Bank\\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\\RWA\\code" # 20250403 George commented out Francesca/Joseph's code

code_dir = current_dir # 20250403 George added

input_dir = code_dir + '\\..\\' + "Inputs" #JT
Output_dir = code_dir + '\\..\\' + "Outputs" #JT

Jason_Input_dir = code_dir + '\\..\\' + "Jason Dec. 2022 RWA Inputs"

Jason_Output_dir = code_dir + '\\..\\' + "Replicated Dec 2022 Outputs"


# Input and Proces Control

# Compare EQB Dec. 2022 with Dec.2024

In [ ]:
# Read eqb_rwa_original_S5_2024.xlsx from Outputs

eqb_rwa_original_S5_202412 = pd.read_excel(Output_dir + '\\eqb_rwa_original_S5_202412.xlsx', sheet_name='RWA_Data')

# Read eqb_rwa_original_S5_202212.xlsx from Jason Outputs

eqb_rwa_original_S5_202212 = pd.read_excel(Jason_Output_dir + '\\eqb_rwa_original_S5_202212.xlsx', sheet_name='RWA_Data')


In [ ]:
# Add a column Reporting_Date to eqb_rwa_original_S5_202412 to have value '2024-12-31'

eqb_rwa_original_S5_202412['Reporting_Date'] = pd.to_datetime('2024-12-31')

# Add a column Reporting_Date to eqb_rwa_original_S5_202212 to have value '2022-12-31'

eqb_rwa_original_S5_202212['Reporting_Date'] = pd.to_datetime('2022-12-31')



In [14]:
# Concatenate eqb_rwa_original_S5_202412 and eqb_rwa_original_S5_202212

eqb_rwa_original_S5_all = pd.concat([eqb_rwa_original_S5_202412, eqb_rwa_original_S5_202212], ignore_index=True)

# Save the data set to a csv file

eqb_rwa_original_S5_all.to_csv(Output_dir + '\\eqb_rwa_original_S5_all.csv', index=False)



In [15]:
# Merge eqb_rwa_original_S5_202412 and eqb_rwa_original_S5_202212 on 'Loan_Number' and keep identifiers from both dataframes

df_Merge = pd.merge(eqb_rwa_original_S5_202412, eqb_rwa_original_S5_202212, on='Loan_Number', how='outer', suffixes=('_2024', '_2022'), indicator=True)

# Save Data to Parquet File

df_Merge.to_parquet(Output_dir + '\\RWA_Data_Merge_2024_2022.parquet', index=False)

# Save Data to CSV File

df_Merge.to_csv(Output_dir + '\\RWA_Data_Merge_2024_2022.csv', index=False)

In [16]:
# Create a pivot table with rows as MRS_Bin_PD_2022 and columns as MRS_Bin_PD_2024, and values as count of Loan_Number

# Fill missing values in MRS_Bin_PD_2024 with 'Closed'

df_Merge['MRS_Bin_PD_2024'] = df_Merge['MRS_Bin_PD_2024'].fillna('Closed')

# Fill missing values in MRS_Bin_PD_2022 with 'New'

df_Merge['MRS_Bin_PD_2022'] = df_Merge['MRS_Bin_PD_2022'].fillna('New')

pivot_table = pd.pivot_table(df_Merge,
                             index='MRS_Bin_PD_2022',
                             columns='MRS_Bin_PD_2024',
                             values='Loan_Number',
                             aggfunc='count',
                                fill_value=0, 
                                margins=True,
                                margins_name='Total',
                                dropna=False)


